# 📊 DataFrame Introduction: Structured Data Processing

**Time to complete:** 30 minutes  
**Difficulty:** Beginner  
**Prerequisites:** Spark Session and RDD basics

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- ✅ Understand what DataFrames are and why they're important
- ✅ Learn how to create DataFrames from different sources
- ✅ Know how to inspect and explore DataFrame structure
- ✅ Perform basic DataFrame operations (select, filter, sort)
- ✅ Understand the relationship between DataFrames and RDDs
- ✅ Be ready to work with structured data in PySpark

---

## 🔍 What is a DataFrame?

A **DataFrame** is Spark's primary abstraction for working with **structured data**. Think of it as:

- **A distributed table** with named columns
- **Schema-aware** data with defined data types
- **Higher-level API** compared to RDDs
- **Optimized for analytics** and SQL-like operations

### Why DataFrames Matter:
- Built on top of RDDs but easier to use
- Automatic optimization via Catalyst optimizer
- Support for SQL queries
- Integration with pandas, databases, and file formats

**DataFrames are what you'll use for 90% of your PySpark work!**

## 🏗️ Creating DataFrames

There are many ways to create DataFrames in PySpark:

In [ ]:
# First, create a Spark Session (if not already created)
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("DataFrame_Introduction") \
    .master("local[*]") \
    .getOrCreate()

print(f"✅ Spark Session ready - Version: {spark.version}")


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/20 12:07:56 WARN Utils: Your hostname, Devs-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.107 instead (on interface en0)
25/12/20 12:07:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/20 12:07:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/20 12:07:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


✅ Spark Session ready - Version: 4.1.0


### Method 1: From Python Lists

In [ ]:
# Create DataFrame from Python list of tuples
data = [
    ("Alice", 25, "Engineering", 75000),
    ("Bob", 30, "Sales", 65000),
    ("Charlie", 35, "Engineering", 85000),
    ("Diana", 28, "HR", 55000),
    ("Eve", 32, "Sales", 70000)
]

# Define column names
columns = ["name", "age", "department", "salary"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

print("✅ DataFrame created from Python lists")
print(f"Type: {type(df)}")
print(f"Number of rows: {df.count()}")
print(f"Number of columns: {len(df.columns)}")


### Method 2: From Python Dictionaries

In [ ]:
# Create DataFrame from list of dictionaries
dict_data = [
    {"name": "Frank", "age": 29, "department": "Marketing", "salary": 60000},
    {"name": "Grace", "age": 31, "department": "Finance", "salary": 72000},
    {"name": "Henry", "age": 27, "department": "IT", "salary": 68000}
]

# Create DataFrame (column names inferred from dict keys)
df_dict = spark.createDataFrame(dict_data)

print("✅ DataFrame created from dictionaries")
print(f"Columns: {df_dict.columns}")
df_dict.show()


### Method 3: From RDDs

In [ ]:
# Create RDD first
rdd_data = spark.sparkContext.parallelize([
    ("Ivy", 33, "Operations", 71000),
    ("Jack", 26, "Engineering", 78000)
])

# Convert RDD to DataFrame
df_from_rdd = rdd_data.toDF(["name", "age", "department", "salary"])

print("✅ DataFrame created from RDD")
print("RDD → DataFrame conversion:")
df_from_rdd.show()

# Combine with existing DataFrame
combined_df = df.union(df_from_rdd)
print(f"\nCombined DataFrame has {combined_df.count()} rows")


### Method 4: From External Files

In [ ]:
# Create sample CSV data for demonstration
csv_data = """name,age,department,salary
Kevin,29,Data Science,80000
Luna,31,Product,75000
Mike,28,Engineering,82000"""

with open("sample_data.csv", "w") as f:
    f.write(csv_data)

# Read CSV file
df_csv = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("sample_data.csv")

print("✅ DataFrame created from CSV file")
df_csv.show()
print(f"Schema: {df_csv.dtypes}")


## 📋 DataFrame Inspection

Let's explore how to examine DataFrame structure and content:

In [ ]:
# Use our main DataFrame for exploration
print("🔍 DataFrame Inspection:")
print(f"Shape: {df.count()} rows × {len(df.columns)} columns")
print(f"Columns: {df.columns}")
print(f"Data types: {df.dtypes}")

# Display data
print("\n📊 First 3 rows:")
df.show(3)

print("📋 Schema information:")
df.printSchema()


### Understanding Schema

The **schema** defines the structure of your DataFrame:
- **Column names** and their **data types**
- **Nullable** indicates if null values are allowed
- **Automatically inferred** or **explicitly defined**

Schemas ensure **type safety** and enable **optimizations**.

## 🎯 Basic DataFrame Operations

Now let's perform common operations on DataFrames:

### Selecting Columns

In [ ]:
# Select specific columns
print("🎯 Column Selection:")

# Method 1: Using column names
name_dept = df.select("name", "department")
print("Names and departments:")
name_dept.show()

# Method 2: Using column expressions
from pyspark.sql.functions import col
name_and_salary = df.select(col("name"), col("salary"))
print("\nNames and salaries:")
name_and_salary.show()


### Filtering Rows

In [ ]:
# Filter rows based on conditions
print("🔍 Row Filtering:")

# Simple filter
engineers = df.filter(col("department") == "Engineering")
print("Engineering team:")
engineers.show()

# Multiple conditions
high_earners = df.filter((col("salary") > 70000) & (col("age") < 35))
print("\nHigh earners under 35:")
high_earners.show()

# Age range filter
young_professionals = df.filter(col("age").between(25, 30))
print("\nYoung professionals (25-30):")
young_professionals.show()


### Sorting Data

In [ ]:
# Sort DataFrame by columns
print("📈 Data Sorting:")

# Sort by salary (descending)
by_salary = df.orderBy(col("salary").desc())
print("Sorted by salary (highest first):")
by_salary.show()

# Sort by department, then by age
by_dept_age = df.orderBy(["department", "age"])
print("\nSorted by department and age:")
by_dept_age.show()

# Sort by age (ascending) and salary (descending)
complex_sort = df.orderBy(col("age").asc(), col("salary").desc())
print("\nSorted by age (asc) and salary (desc):")
complex_sort.show()


### Adding and Modifying Columns

In [ ]:
# Add new columns and modify existing ones
print("➕ Column Operations:")

# Add bonus column (10% of salary)
with_bonus = df.withColumn("bonus", col("salary") * 0.1)
print("Added bonus column:")
with_bonus.select("name", "salary", "bonus").show()

# Add experience level based on age
from pyspark.sql.functions import when
with_experience = df.withColumn(
    "experience_level",
    when(col("age") < 28, "Junior")
    .when(col("age") < 33, "Mid-level")
    .otherwise("Senior")
)
print("\nAdded experience level:")
with_experience.select("name", "age", "experience_level").show()

# Rename column
renamed_df = df.withColumnRenamed("salary", "annual_salary")
print("\nRenamed 'salary' to 'annual_salary':")
print(f"Columns: {renamed_df.columns}")


## 📊 Aggregation Operations

In [ ]:
# Group and aggregate data
from pyspark.sql.functions import avg, sum, count, max, min

print("📊 Aggregation Operations:")

# Group by department and calculate statistics
dept_stats = df.groupBy("department").agg(
    count("name").alias("employee_count"),
    avg("salary").alias("avg_salary"),
    max("salary").alias("max_salary"),
    min("age").alias("min_age")
)

print("Department statistics:")
dept_stats.show()

# Overall statistics
overall_stats = df.agg(
    count("*").alias("total_employees"),
    avg("salary").alias("avg_company_salary"),
    sum("salary").alias("total_salary_cost"),
    max("age").alias("oldest_employee"),
    min("age").alias("youngest_employee")
)

print("\nCompany-wide statistics:")
overall_stats.show()


## 🔄 DataFrames vs RDDs

Understanding when to use each:

In [ ]:
# Create equivalent RDD and DataFrame
sample_data = [("Alice", 25), ("Bob", 30), ("Charlie", 35)]

# RDD approach
rdd = spark.sparkContext.parallelize(sample_data)
rdd_result = rdd.filter(lambda x: x[1] > 26).map(lambda x: (x[0], x[1] * 2)).collect()

# DataFrame approach
df_sample = spark.createDataFrame(sample_data, ["name", "age"])
df_result = df_sample.filter(col("age") > 26).withColumn("doubled_age", col("age") * 2)

print("🔄 RDD vs DataFrame Comparison:")
print(f"RDD result: {rdd_result}")
print("DataFrame result:")
df_result.show()

print("\nKey Differences:")
print("- RDDs: Low-level, manual optimization, all data types")
print("- DataFrames: High-level, automatic optimization, structured data")
print("- Choose DataFrames for 90% of analytics workloads!")


## 🧹 Cleaning Up

In [ ]:
# Clean up sample files
import os
if os.path.exists("sample_data.csv"):
    os.remove("sample_data.csv")

print("🧹 Cleanup completed")

# Note: Spark Session will be cleaned up automatically in Jupyter
# In production code, use: spark.stop()


## 🎯 Key Takeaways

### What You Learned:
- ✅ **DataFrames** are Spark's primary tool for structured data
- ✅ **Schema** defines column names and data types
- ✅ **Multiple creation methods** from lists, dicts, RDDs, files
- ✅ **Rich operations** for selection, filtering, sorting, aggregation
- ✅ **Higher-level API** than RDDs with automatic optimization

### DataFrame Advantages:
- 🔸 **Easier to use** than RDDs for structured data
- 🔸 **Automatic optimization** via Catalyst optimizer
- 🔸 **SQL support** for familiar query syntax
- 🔸 **Rich ecosystem** of built-in functions
- 🔸 **Better performance** for analytical workloads

### Common Operations:
- 🔸 `select()` - Choose columns
- 🔸 `filter()` - Filter rows by conditions
- 🔸 `orderBy()` - Sort data
- 🔸 `groupBy().agg()` - Group and aggregate
- 🔸 `withColumn()` - Add/modify columns
- 🔸 `show()` - Display results

### When to Use DataFrames:
- ✅ **Structured data** with defined schema
- ✅ **SQL-like operations** (filtering, aggregation)
- ✅ **File-based data** (CSV, JSON, Parquet)
- ✅ **Analytics and reporting** workloads

---

## 🚀 Next Steps

Now that you understand DataFrames, you're ready for:

1. **Transformations vs Actions Deep Dive** - Understanding lazy evaluation
2. **DataFrame Mastery** - Advanced operations and window functions
3. **Spark SQL** - Querying DataFrames with SQL

---

**🎉 Congratulations! You now have the foundation for working with structured data in PySpark!**